이번 분석의 주 함수.
- 클래스 정의이기 때문에 한 블럭에 주석을 달아 분석.

In [6]:
import math

# class declaration
class SentiAnalyzer:
    
    # Make the method signature to accept "sentidata" and "word"
    # --> ?? 무슨 소린지 모르겠다.
    
    # 처음 instantiated된 class가 실행될 때 마다 출력되는 생성자(constructor)
    def __init__(self):
        print('This is a senti analyzer')
     
    # 특정 인덱스의 단어의 긍정적일 확률과 부정적일 확률을 출력.
        # idxWord: 28,624개의 단어 중 분석하고 싶은 단어의 인덱스.
    def probWordPositiveAndNegative(self, sentidata, word, idxWord):
        pointedWord = word[idxWord] # idxWord로 선택한 단어.
        reviews = [int(float(row[-1])) for row in sentidata] # sentidata중 review data가 있는 마지막 column만 출력.
        occurrence = [int(float(row[idxWord])) for row in sentidata] # sentidata 중 pointedWord 열을 추출.
        
        # pointedWord의 열 중 긍정적인 리뷰의 개수.
        positive = 0
        for i in range(len(occurrence)): # range는 198.
            # 긍정적인 리뷰 중에 해당 단어가 있다면 더하기.
            positive = positive + occurrence[i] * reviews[i]
        
        # 전체 198개의 리뷰 중 긍정적인 리뷰의 개수.
        numPositiveReviews = sum(reviews)
        
        # pointedWord의 열 중 부정적인 리뷰의 개수.
        # revies[i]가 0이면 부정적인 리뷰.
        negative = 0
        for i in range(len(occurrence)):
            # 부정적인 리뷰 중에 해당 단어가 있다면 더하기.
            negative = negative + occurrence[i] * (1-reviews[i])
            
        rowCount = len(sentidata) # 198.
        positiveProb = float(positive) / float(numPositiveReviews)
        negativeProb = float(negative) / float(rowCount - numPositiveReviews)
        
        if positiveProb == 0:
            positiveProb = 0.00001
        if negativeProb == 0:
            negativeProb = 0.00001
        return pointedWord, positiveProb, negativeProb
    
    # 전체 198개의 리뷰 중에 긍정적인 리뷰와 부정적인 리뷰인 확률 출력.
    def probPositiveAndNegative(self, sentidata):
        positive = sum([int(float(row[-1])) for row in sentidata]) # 위의 sum(reviews[i])와 같다.
        numReviews = len(sentidata) # 위의 rowCount와 같다.
        negative = numReviews - positive
        
        positiveProb = float(positive) / float(numReviews)
        negativeProb = float(negative) / float(numReviews)
        
        return positiveProb, negativeProb
    
    # idx번째 리뷰에서 사용된 단어와 그 단어의 인덱스를 출력.
    def findUsedWords(self,sentidata, word, idx):
        temp = [int(float(x)) for x in sentidata[idx[:-1]]] # idx번째 리뷰를 형 변환.
        idxUsedWords = [index for index, value in enumerate(temp) if value == 1] # idx번째 리뷰에 사용된 단어들의 인덱스 리스트.
        usedWords = [word[idx] for idx in idxUsedWords] # word중 idx번째 리뷰에 사용된 단어 리스트.
        
        return idxUsedWords, usedWords
    
    # 위의 함수들을 사용해 분석한 내용을 출력.
    def runAnalysis(self, sentidata, word, idxReview):
        # 로그를 사용하는 이유: 기본적으로 로그는 큰 수를 같은 비율의 작은 수로 바꿔주는 것. 복잡한 계산을 간편하게 만들어준다.
        probLogPositive = 0
        probLogNegative = 0
        idxUsedWords, usedWords = self.findUsedWords(sentidata, word, idxReview) # idx번째 리뷰에 사용된 단어와 그 단어의 인덱스 리스트.
        
        # idx번째 리뷰에 들어있는 단어 개수만큼 for loop 돌린다.
        for i in range(len(idxUsedWords)):
            idxWord = idxUsedWords[i]
            # i번째 단어마다 긍정, 부정 확률을 구한다.
            pointedWord, positive, negative = self.probWordPositiveAndNegative(sentidata, word, idxWord)
            probLogPositive += math.log(positive)
            probLogNegative += math.log(negative)
            
            if probLogPositive > probLogNegative:
                sentiment = 'Positive'
                print('Positive')
            else:
                sentiment = 'Negative'
                print('Negative')
                
            return probLogPositive, probLogNegative, sentiment